In [1]:
import pandas as pd

from helper import *
from snp_func import *

# Do assignments
a = assign_var('snp')
for v in a:
    exec(v)

from ib_insync import *
util.startLoop()
ib = get_connected('snp', 'live')

# Code to set up harvest quotes
# ______________________________
p = util.df(ib.portfolio()) # portfolio table

# extract option contract info from portfolio table
dfp = pd.concat([p, util.df([c for c in p.contract])[util.df([c for c in p.contract]).columns[:7]]], axis=1).iloc[:, 1:]
dfp = dfp.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

dfp = dfp.assign(dte=[get_dte(d) for d in dfp.expiration])

# get the latest prices
contracts = ib.qualifyContracts(*[Contract(conId=c) for c in dfp.conId])

tickers = ib.reqTickers(*contracts)
optPrices = {t.contract.conId: t.marketPrice() for t in tickers} # {conId: optPrice}

# make the expPrice to be the price minimum of df.marketPrice*0.9 and hvstPrice()
dfp1 = dfp.assign(hvstPrice=[get_prec(min(hvstPricePct(d),mp*0.9),0.01) for d, mp in zip(dfp.dte, dfp.marketPrice)])
dfp1 = dfp1.loc[:, ['position', 'marketPrice', 'averageCost', 'secType', 'conId', 'symbol', 'expiration', 'strike', 'right', 'dte', 'hvstPrice']]

dfp1 = dfp1.assign(order=[LimitOrder(action='BUY', 
                                     totalQuantity=-position, lmtPrice=hvstPrice) 
                          for position, hvstPrice in zip(dfp1.position, dfp1.hvstPrice)])

In [2]:
dfp1

,position,marketPrice,averageCost,secType,conId,symbol,expiration,strike,right,dte,hvstPrice,order
0,-5.0,0.075106,123.627580,OPT,358200413,ABBV,20190503,75.5,P,4,0.07,"LimitOrder(action='BUY', totalQuantity=5.0, lm..."
1,-1.0,0.168317,74.328200,OPT,358205647,AMGN,20190503,160.0,P,4,0.15,"LimitOrder(action='BUY', totalQuantity=1.0, lm..."
2,-1.0,0.357444,104.327800,OPT,358205659,AMGN,20190503,165.0,P,4,0.32,"LimitOrder(action='BUY', totalQuantity=1.0, lm..."
3,-1.0,0.207513,44.328600,OPT,353820937,AMGN,20190517,150.0,P,18,0.19,"LimitOrder(action='BUY', totalQuantity=1.0, lm..."
4,-1.0,0.380628,61.328400,OPT,353820951,AMGN,20190517,155.0,P,18,0.34,"LimitOrder(action='BUY', totalQuantity=1.0, lm..."
5,-1.0,0.656380,86.328100,OPT,353820964,AMGN,20190517,160.0,P,18,0.49,"LimitOrder(action='BUY', totalQuantity=1.0, lm..."
6,-1.0,1.096313,125.327600,OPT,353820974,AMGN,20190517,165.0,P,18,0.49,"LimitOrder(action='BUY', totalQuantity=1.0, lm..."
7,-9.0,0.020262,37.628711,OPT,358219353,BMY,20190503,43.0,P,4,0.02,"LimitOrder(action='BUY', totalQuantity=9.0, lm..."
8,-8.0,0.250249,25.628863,OPT,358229911,CVS,20190503,49.0,P,4,0.23,"LimitOrder(action='BUY', totalQuantity=8.0, lm..."
9,-8.0,0.308744,33.628763,OPT,358229897,CVS,20190503,49.5,P,4,0.28,"LimitOrder(action='BUY', totalQuantity=8.0, lm..."


In [ ]:
df_hvstTrades

In [ ]:
df_hvstTrades = pd.concat([pd.DataFrame(harvestTrades), util.df(harvestTrades)], axis=1)
df_hvstTrades.columns=[['trade']+list(df_hvstTrades)[1:]]
df_hvstTrades.to_pickle(fspath+'_hvstTradesDone.pickle')

In [19]:
# Code to set up sowing orders
# ____________________________
dft = pd.read_pickle(fspath+'_targetopts.pickle')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
grp_opts(dft)

,optId,symbol,right,expiration,dte,strike,undPrice,lo52,hi52,Fall,Rise,loFall,hiRise,Std,loStd,hiStd,lotsize,optPrice,optMargin,rom,remqty,expPrice,expRom
0,358972719,ABBV,P,20190510,12,75.0,79.34,75.77,125.86,30.330,27.300,49.010,106.640,2.092390,75.155219,85.617171,100,0.34,190,5.442982,1,0.374,5.987281
1,358200465,ABBV,P,20190503,5,75.0,79.34,75.77,125.86,22.670,21.760,56.670,101.100,2.046526,75.246948,85.479578,100,0.14,169,6.047337,1,0.154,6.652071
2,358200382,ABBV,P,20190503,5,74.5,79.34,75.77,125.86,22.670,21.760,56.670,101.100,2.046526,75.246948,85.479578,100,0.12,150,5.840000,1,0.132,6.424000
3,316297786,ABT,P,20190621,54,52.5,78.11,54.32,80.74,9.480,14.990,68.630,93.100,4.063251,69.983499,90.299752,100,0.02,29,0.466156,6,0.070,1.631545
4,316297781,ABT,P,20190621,54,50.0,78.11,54.32,80.74,9.480,14.990,68.630,93.100,4.063251,69.983499,90.299752,100,0.01,23,0.293881,6,0.070,2.057166
5,315853188,ACN,P,20190621,54,125.0,179.95,132.63,181.41,38.140,35.740,141.810,215.690,12.923375,154.103251,218.720124,100,0.10,83,0.814369,2,0.110,0.895805
6,361266108,AGN,P,20190524,26,124.0,141.49,125.84,197.00,47.130,35.190,94.360,176.680,6.988571,127.512858,162.455713,100,0.87,224,5.452438,3,0.957,5.997682
7,334373991,AGN,P,20190517,19,125.0,141.49,125.84,197.00,39.570,35.050,101.920,176.540,5.579733,130.330534,158.229199,100,0.60,217,5.311666,3,0.660,5.842833
8,360120924,AGN,P,20190510,12,125.0,141.49,125.84,197.00,37.690,28.600,103.800,170.090,3.992747,133.504507,153.468240,100,0.56,207,8.228663,3,0.616,9.051530
9,337639876,AIG,P,20190517,19,36.0,45.19,36.16,65.05,15.150,7.630,30.040,52.820,1.354729,42.480541,49.254188,100,0.08,38,4.044321,10,0.088,4.448753


2019-04-29 08:19:50,399 ib_insync.wrapper INFO orderStatus: Trade(contract=Contract(secType='OPT', conId=289157898, symbol='JPM', lastTradeDateOrContractMonth='20190621', strike=90.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='JPM   190621P00090000', tradingClass='JPM'), order=LimitOrder(softDollarTier=SoftDollarTier(), orderId=144131, clientId=1, permId=174688026, action='SELL', totalQuantity=100.0, lmtPrice=0.09, auxPrice=0.0, tif='DAY', ocaType=3, openClose='', eTradeOnly=False, firmQuoteOnly=False, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U8898867', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(status='Inactive', remaining=100.0, permId=174688026, clientId=1), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 4, 29, 6, 17, 51, 232152, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.dateti

2019-04-29 08:19:50,478 ib_insync.wrapper INFO orderStatus: Trade(contract=Contract(secType='OPT', conId=303303274, symbol='UTX', lastTradeDateOrContractMonth='20190621', strike=95.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='UTX   190621P00095000', tradingClass='UTX'), order=LimitOrder(softDollarTier=SoftDollarTier(), orderId=144190, clientId=1, permId=174688064, action='SELL', totalQuantity=100.0, lmtPrice=0.07, auxPrice=0.0, tif='DAY', ocaType=3, openClose='', eTradeOnly=False, firmQuoteOnly=False, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U8898867', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(status='Inactive', remaining=100.0, permId=174688064, clientId=1), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 4, 29, 6, 17, 52, 321240, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.dateti

2019-04-29 08:19:50,502 ib_insync.wrapper INFO orderStatus: Trade(contract=Contract(secType='OPT', conId=315291554, symbol='PEP', lastTradeDateOrContractMonth='20190621', strike=95.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='PEP   190621P00095000', tradingClass='PEP'), order=LimitOrder(softDollarTier=SoftDollarTier(), orderId=144164, clientId=1, permId=174688047, action='SELL', totalQuantity=100.0, lmtPrice=0.09, auxPrice=0.0, tif='DAY', ocaType=3, openClose='', eTradeOnly=False, firmQuoteOnly=False, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U8898867', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(status='Inactive', remaining=100.0, permId=174688047, clientId=1), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 4, 29, 6, 17, 52, 285336, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.dateti

2019-04-29 08:19:50,537 ib_insync.wrapper INFO orderStatus: Trade(contract=Contract(secType='OPT', conId=316445213, symbol='NVDA', lastTradeDateOrContractMonth='20190621', strike=330.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='NVDA  190621C00330000', tradingClass='NVDA'), order=LimitOrder(softDollarTier=SoftDollarTier(), orderId=144275, clientId=1, permId=174688115, action='SELL', totalQuantity=100.0, lmtPrice=0.07, auxPrice=0.0, tif='DAY', ocaType=3, openClose='', eTradeOnly=False, firmQuoteOnly=False, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U8898867', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(status='Inactive', remaining=100.0, permId=174688115, clientId=1), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 4, 29, 6, 17, 54, 478474, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.dat

...

2019-04-29 08:19:53,043 ib_insync.wrapper WARNING Canceled order: Trade(contract=Contract(secType='OPT', conId=361955635, symbol='WMT', lastTradeDateOrContractMonth='20190517', strike=111.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='WMT   190517C00111000', tradingClass='WMT'), order=LimitOrder(softDollarTier=SoftDollarTier(), orderId=144303, clientId=1, permId=174688135, action='SELL', totalQuantity=100.0, lmtPrice=0.22, auxPrice=0.0, tif='DAY', ocaType=3, openClose='', eTradeOnly=False, firmQuoteOnly=False, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U8898867', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(status='Cancelled', remaining=100.0, permId=174688135, clientId=1), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 4, 29, 6, 17, 54, 527343, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetim

2019-04-29 08:19:53,054 ib_insync.wrapper ERROR Error 201, reqId 144207: Order rejected - reason:Order Reject: We are unable to compute a value for this contract. 

2019-04-29 08:19:53,055 ib_insync.wrapper WARNING Canceled order: Trade(contract=Contract(secType='OPT', conId=362340281, symbol='AXP', lastTradeDateOrContractMonth='20190510', strike=124.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='AXP   190510C00124000', tradingClass='AXP'), order=LimitOrder(softDollarTier=SoftDollarTier(), orderId=144207, clientId=1, permId=174688072, action='SELL', totalQuantity=100.0, lmtPrice=0.07, auxPrice=0.0, tif='DAY', ocaType=3, openClose='', eTradeOnly=False, firmQuoteOnly=False, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U8898867', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(status='Cancelled', remaining=100.0, permId=174688072, clientId=1), fills=[], log

2019-04-29 08:19:53,063 ib_insync.wrapper INFO orderStatus: Trade(contract=Contract(secType='OPT', conId=362352465, symbol='CL', lastTradeDateOrContractMonth='20190503', strike=79.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='CL    190503C00079000', tradingClass='CL'), order=LimitOrder(softDollarTier=SoftDollarTier(), orderId=144215, clientId=1, permId=174688076, action='SELL', totalQuantity=100.0, lmtPrice=0.07, auxPrice=0.0, tif='DAY', ocaType=3, openClose='', eTradeOnly=False, firmQuoteOnly=False, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U8898867', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(status='Inactive', remaining=100.0, permId=174688076, clientId=1), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 4, 29, 6, 17, 53, 365449, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime

2019-04-29 08:19:53,071 ib_insync.wrapper WARNING Canceled order: Trade(contract=Contract(secType='OPT', conId=362382923, symbol='GE', lastTradeDateOrContractMonth='20190607', strike=6.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='GE    190607P00006000', tradingClass='GE'), order=LimitOrder(softDollarTier=SoftDollarTier(), orderId=144112, clientId=1, permId=174688013, action='SELL', totalQuantity=100.0, lmtPrice=0.07, auxPrice=0.0, tif='DAY', ocaType=3, openClose='', eTradeOnly=False, firmQuoteOnly=False, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U8898867', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(status='Cancelled', remaining=100.0, permId=174688013, clientId=1), fills=[], log=[TradeLogEntry(time=datetime.datetime(2019, 4, 29, 6, 17, 51, 199240, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.da

2019-04-29 08:19:53,207 ib_insync.wrapper ERROR Error 201, reqId 144169: Order rejected - reason:Order Reject: We are unable to compute a value for this contract. 

2019-04-29 08:19:53,208 ib_insync.wrapper WARNING Canceled order: Trade(contract=Contract(secType='OPT', conId=362439557, symbol='PFE', lastTradeDateOrContractMonth='20190607', strike=33.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='PFE   190607P00033000', tradingClass='PFE'), order=LimitOrder(softDollarTier=SoftDollarTier(), orderId=144169, clientId=1, permId=174688051, action='SELL', totalQuantity=100.0, lmtPrice=0.07, auxPrice=0.0, tif='DAY', ocaType=3, openClose='', eTradeOnly=False, firmQuoteOnly=False, volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U8898867', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(status='Cancelled', remaining=100.0, permId=174688051, clientId=1), fills=[], log=

In [21]:
# qualify contracts and prepare to order
contracts = [Contract(conId=c) for c in dft.optId]
cblks = [contracts[i: i+blk] for i in range(0, len(contracts), blk)]
qc = [c for q in [ib.qualifyContracts(*c) for c in cblks] for c in q]

sell_order = [LimitOrder(action='SELL', totalQuantity=1, lmtPrice=expPrice) for q, expPrice in zip(dft.lotsize, dft.expPrice)]

co = list(zip(qc, sell_order))
coblks = [co[i: i+blk] for i in range(0, len(co), blk)]

util.logToConsole()

2019-04-29 08:22:26,615 ib_insync.client WARNING Started to throttle requests
2019-04-29 08:22:27,625 ib_insync.client WARNING Stopped to throttle requests
2019-04-29 08:22:27,689 ib_insync.client WARNING Started to throttle requests
2019-04-29 08:22:28,687 ib_insync.client WARNING Stopped to throttle requests
2019-04-29 08:22:29,194 ib_insync.client WARNING Started to throttle requests
2019-04-29 08:22:30,203 ib_insync.client WARNING Stopped to throttle requests
2019-04-29 08:22:31,144 ib_insync.client WARNING Started to throttle requests
2019-04-29 08:22:32,156 ib_insync.client WARNING Stopped to throttle requests
2019-04-29 08:22:32,705 ib_insync.client WARNING Started to throttle requests
2019-04-29 08:22:33,703 ib_insync.client WARNING Stopped to throttle requests


In [ ]:
[(co[0], co[1]) for coblk in coblks for co in coblk]

In [ ]:
df_trades = pd.concat([pd.DataFrame(trades), util.df(trades)], axis=1)
df_trades.columns=[['trade']+list(df_trades)[1:]]
df_trades.to_pickle(fspath+'_tradesDone.pickle')

df_trades.head()

In [ ]:
# Execute the trades
def xcute(ib, fspath):
    '''Executes trades
    Args:
       (ib) as connection object
       (fspath) as string with path to pickles
    Returns:
       None. But pickles _tradesdone.pickle'''
    
    # read from the targets
    dft = pd.read_pickle(fspath+'_targetopts.pickle')